#### Reading JSON Config file

In [ ]:
# config.json can be like this to get the csv names and columns that will be used in script
{
    "file1":
        {
            "filename": "input.csv",
            "columns":
                {
                    "column1" :  "column1",
                    "column2" :  "column2"
                }
        }
}

In [11]:
import sys
import os
import json
import time

def check_files():
    cf = open("config.json")
    data = json.load(cf)
    df_filename = data['file1']['filename']
    df_columns  = data['file1']['columns']

    if os.path.exists(df_filename) :
        print("The file indicated in the config.json was read successfully. ")
        return  df_filename,  df_columns
    else:
        print("The file indicated in the config.json is not in the related folder.")
        time.sleep(5)
        sys.exit()

In [12]:
check_files()

The file indicated in the Config.json was read successfully. 


('input.csv', {'column1': 'column1', 'column2': 'column2'})

#### Finding the Path of the Current Python File

In [25]:
# Full path of the current file
import os
import sys
full_path = os.path.abspath('')
full_path

'C:\\Users\\enes.teymir\\OneDrive - sabancidx.com\\Desktop\\Python_Files'

PyInstaller can bundle your script and all its dependencies into a single executable named myscript ( myscript.exe in Windows).
When your program is not frozen, the standard Python variable __file__ is the full path to the script now executing. When a bundled app starts up, the bootloader sets the sys.frozen attribute and stores the absolute path to the bundle folder in sys._MEIPASS. For a one-folder bundle, this is the path to that folder, wherever the user may have put it. For a one-file bundle, this is the path to the _MEIxxxxxx

In [ ]:
file_path = os.path.abspath(os.path.dirname(__file__))        # the absolute path of the directory where the program resides.
parent_path = os.path.join(os.path.dirname(__file__), '..')   # the parent directory of the directory where program resides.

In [33]:
# This will check the _MEIPASS to get the path of folder. If it is not found then getattr will return ''.
meipass = getattr(sys, "_MEIPASS", '')
full_path = os.path.abspath(meipass)
full_path

'C:\\Users\\enes.teymir\\OneDrive - sabancidx.com\\Desktop\\Python_Files'

In [ ]:
# this will give the bundle path, it will check _MEIPASS variable at first, if it is not found then will get the path from the __file__ variable
# os.path.dirname() returns the directory name of pathname path
bundle_dir = getattr(sys, '_MEIPASS', os.path.abspath(os.path.dirname(__file__)))

In [35]:
# If we apply it to our file's path, we can see the file name is not included in the path's end
getattr(sys, '_MEIPASS', os.path.abspath(os.path.dirname(full_path)))

'C:\\Users\\enes.teymir\\OneDrive - sabancidx.com\\Desktop'

In [ ]:
# If we want to get the path to spesific file in the bundle app folder, use this
path_to_file = os.path.abspath(os.path.join(bundle_dir, 'filename'))

In [ ]:
# special variables can be seen in global()
print(globals())

Python files are called modules and they are identified by the .py file extension. A module can define functions, classes, and variables.
So when the interpreter runs a module, the __name__ variable will be set as  __main__ if the module that is being run is the main program.
But if the code is importing the module from another module, then the __name__  variable will be set to that module’s name.
The variable __name__ for the file/module that is run will be always __main__. But the __name__ variable for all other modules that are being imported will be set to their module's name.
For more : https://www.freecodecamp.org/news/if-name-main-python-example/

In [5]:
print(__name__)

__main__


#### After checking files, creating sqlite db and related tables , and put the csv files in the folder. We can run our script

In [ ]:
if __name__ == '__main__':                             # used to execute some code only if the file was run directly, and not imported.

    df_filename, df_columns = check_files()            # get the file name and columns name from check function
    bundle_dir = getattr(sys, '_MEIPASS', os.path.abspath(os.path.dirname(__file__)))
    path_to_db = os.path.abspath(os.path.join(bundle_dir, 'dbname.db'))
    path_to_pkl = os.path.abspath(os.path.join(bundle_dir, 'iForest.pkl'))
    iforest = pickle.load(open(path_to_pkl, 'rb'))
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)    # You can write the files to sqlite db. Create a db on python folder and create tables. Then define the path to db. U can use DB Browser for SQLite
    loc_df = pd.read_sql_table('loc_df', engine)
    new_df = pd.read_csv(df_filename)               # read the file as defined in config file
    new_df = new_df[list(df_columns.values())]      # make column names as defined in json file
    new_df.columns = list(df_columns.keys())        # change column names the ones that will be used in model

else:
    print("File was imported ! You should execute it directly to get the results !")

#### Creating a SPEC File : Pysinstaller create this file automatically when you run it. You can modify it.

In [ ]:
# -*- mode: python -*-
options = [ ('v', None, 'OPTION')]
block_cipher = None

a = Analysis(['python_file_name.py'],                               # write the file name with .py
             pathex=[],
             binaries=[],
             datas=[('iforest.pkl', '.'), ('dbname.db', '.')],     # write the files name here like pickle, database, csv ...  the added data files are copied into the folder with the executable.
             hiddenimports=['sklearn','sklearn.ensemble.IsolationForest'],   # write the hidden imports that raise errors. And remove them from the python script
             hookspath=[],
             runtime_hooks=[],
             excludes=[],
             win_no_prefer_redirects=False,
             win_private_assemblies=False,
             cipher=block_cipher
             )

pyz = PYZ(a.pure, a.zipped_data,
          cipher=block_cipher
          )

exe = EXE(pyz,
          a.scripts,
          a.binaries,
          a.zipfiles,
          a.datas,
          [],
          name='python_file_name',                                   # write the py file's name only
          debug=False,
          bootloader_ignore_signals=False,
          strip=False,
          upx=True,
          upx_exclude=[],
          runtime_tmpdir=None,
          console=True,
          disable_windowed_traceback=False,
          target_arch=None,
          codesign_identity=None,
          entitlements_file=None
          )

In [ ]:
#You can write command like this also to add hidden imports instead of writing in the spec file.
pyinstaller --onefile --hidden-import sklearn.ensemble.IsolationForest  spec_file_name.spec

In [ ]:
# After you prepare the spec file , then can create an exe by this command. Exe will be created in "dist" folder at the end
# You can write "cmd" in the path section in related folder then write this command in terminal
pyinstaller -F --clean  spec_file_name.spec 
# pyinstaller --clean  spec_file_name.spec

In [ ]:
# In Linux,at first env should be activated or global variable should be defined before. And py file path should be defined
source env/bin/activate 
pyinstaller -F --clean  /home/artibir_user/notebooks/Enes/Exe/clustering.spec 